In [3]:
import os
from birdsong.config import config
from birdsong.audiotransform.to_image import AudioPreprocessor


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread

import tensorflow as tf
from tensorflow.keras import datasets


**Preparing the data**

In [5]:
test_input_path = '/Users/aless/code/A54P/birdsong_classifier/data/birdsong_test_input/'
test_output_path = '/Users/aless/code/A54P/birdsong_classifier/data/test_output/'

df = AudioPreprocessor(test_input_path,test_output_path,config.SPECTOGRAM_TYPE, config.OUTPUT_FORMAT)
